In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 7-1. ブロードキャスティングとは？

### 7-1-1. ドット構文（おさらい）

#### コード7-1. シグモイド関数（定義）

In [2]:
function sigmoid(x)
    inv(1 + exp(-x))
end

sigmoid (generic function with 1 method)

#### コード7-2. シグモイド関数の動作確認（引数にスカラーを渡した場合）

In [3]:
sigmoid(-8)

0.0003353501304664781

In [4]:
sigmoid(-6)

0.0024726231566347743

In [5]:
sigmoid(-4)

0.01798620996209156

In [6]:
sigmoid(-2)

0.11920292202211755

In [7]:
sigmoid(0)

0.5

In [8]:
sigmoid(2)

0.8807970779778823

In [9]:
sigmoid(4)

0.9820137900379085

In [10]:
sigmoid(6)

0.9975273768433653

In [11]:
sigmoid(8)

0.9996646498695336

In [12]:
sigmoid(Inf)

1.0

In [13]:
sigmoid(-Inf)

0.0

#### コード7-3. シグモイド関数の動作確認（内包表記とドット構文によるベクタ化の比較）

In [14]:
[sigmoid(x) for x in -8:2:8]

9-element Vector{Float64}:
 0.0003353501304664781
 0.0024726231566347743
 0.01798620996209156
 0.11920292202211755
 0.5
 0.8807970779778823
 0.9820137900379085
 0.9975273768433653
 0.9996646498695336

In [15]:
sigmoid.(-8:2:8)

9-element Vector{Float64}:
 0.0003353501304664781
 0.0024726231566347743
 0.01798620996209156
 0.11920292202211755
 0.5
 0.8807970779778823
 0.9820137900379085
 0.9975273768433653
 0.9996646498695336

#### コード7-4. 代入におけるドット構文の利用

In [16]:
v0 = zeros(5)

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [17]:
r0 = 1:5

1:5

In [18]:
v0 .= r0;  # `for i in axes(v0) v0[i] = r0[i] end` と同等の処理

In [19]:
v0

5-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0
 5.0

In [20]:
v0 .*= [2^x for x=0:4];
v0

5-element Vector{Float64}:
  1.0
  4.0
 12.0
 32.0
 80.0

In [21]:
v1 = v0;  # `v1` にはそのまま代入（＝同じ配列を参照）
v2 = similar(v0); v2 .= v0;  # `v2` として同じサイズ・同じ要素型の新しい配列を確保した上でドット構文による代入

In [22]:
v0[1] = -1;

In [23]:
v0

5-element Vector{Float64}:
 -1.0
  4.0
 12.0
 32.0
 80.0

In [24]:
v1  # === `v0`

5-element Vector{Float64}:
 -1.0
  4.0
 12.0
 32.0
 80.0

In [25]:
v2  # != `v0`

5-element Vector{Float64}:
  1.0
  4.0
 12.0
 32.0
 80.0

#### コード7-5. シグモイド関数による内包表記とドット構文の動作比較（多次元配列、タプル）

In [26]:
A = [-5.0 0.0; 0.0 5.0];

In [27]:
[sigmoid(x) for x in A]

2×2 Matrix{Float64}:
 0.00669285  0.5
 0.5         0.993307

In [28]:
sigmoid.(A)

2×2 Matrix{Float64}:
 0.00669285  0.5
 0.5         0.993307

In [29]:
t = (-Inf, -1, 0, 1, Inf);

In [30]:
[sigmoid(x) for x in t]

5-element Vector{Float64}:
 0.0
 0.2689414213699951
 0.5
 0.7310585786300049
 1.0

In [31]:
sigmoid.(t)

(0.0, 0.2689414213699951, 0.5, 0.7310585786300049, 1.0)

### 7-1-2. ブロードキャスティング

#### コード7-6. ブロードキャスティングの例(1)（ベクトルとスカラーの計算）

In [32]:
μ = 5.0;  # `μ` は "\mu"+<TAB> で入力できます。
σ = 2.0;  # `σ` は "\sigma"+<TAB> で入力できます。
N = 500;

In [33]:
snds = randn(N);  # 要素数 500 の標準正規分布に則った乱数列（ベクトル）

In [34]:
nds1 = [v * σ + μ for v in snds];  # 方法1：内包表記

In [35]:
nds2 = snds .* fill(σ, N) .+ fill(μ, N);  # 方法2：`fill(○, N)` を利用したドット構文（ベクタ化）

In [36]:
nds3 = snds .* σ .+ μ;  # 方法3：ブロードキャスティング

In [37]:
nds1 == nds2 == nds3

true

#### コード7-7. ブロードキャスティングの例(2)（コード7-6. のパフォーマンス比較）

In [38]:
randnds1(N, μ, σ) = [v * σ + μ for v in randn(N)]  # 方法1

randnds1 (generic function with 1 method)

In [39]:
randnds2(N, μ, σ) = randn(N) .* fill(σ, N) .+ fill(μ, N)  # 方法2

randnds2 (generic function with 1 method)

In [40]:
randnds3(N, μ, σ) = randn(N) .* σ .+ μ  # 方法3

randnds3 (generic function with 1 method)

In [41]:
using Random

In [42]:
(Random.seed!(1234);randnds1(N, μ, σ)) == 
(Random.seed!(1234);randnds2(N, μ, σ)) == 
(Random.seed!(1234);randnds3(N, μ, σ))

true

In [43]:
@time randnds1(10000, μ, σ);

  0.000032 seconds (4 allocations: 156.344 KiB)


In [44]:
@time randnds2(10000, μ, σ);

  0.000101 seconds (8 allocations: 312.688 KiB)


In [45]:
@time randnds3(10000, μ, σ);

  0.000049 seconds (4 allocations: 156.344 KiB)


#### コード7-8. ブロードキャスティングの例(3)（行列とベクトルとの演算例）

In [46]:
A1 = zeros(3, 3)

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [47]:
v1 = 1:3

1:3

In [48]:
A1 + v1

LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(3)), must have singleton at dim 2

In [49]:
A1 + v1'

LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(3)), b has dims (Base.OneTo(1), Base.OneTo(3)), mismatch at 1

In [50]:
A1 .+ v1  # `A1 + [v1 v1 v1]` と同等

3×3 Matrix{Float64}:
 1.0  1.0  1.0
 2.0  2.0  2.0
 3.0  3.0  3.0

In [51]:
A1 .+ v1'  # `A1 + [v1'; v1'; v1']` と同等

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 1.0  2.0  3.0
 1.0  2.0  3.0

#### コード7-9. ブロードキャスティングの例(4)（サイズと次元拡張）

In [52]:
A2 = [10 30; 20 40]

2×2 Matrix{Int64}:
 10  30
 20  40

In [53]:
A3 = reshape(1:6, (2, 1, 3))

2×1×3 reshape(::UnitRange{Int64}, 2, 1, 3) with eltype Int64:
[:, :, 1] =
 1
 2

[:, :, 2] =
 3
 4

[:, :, 3] =
 5
 6

In [54]:
size(A2)

(2, 2)

In [55]:
size(A3)

(2, 1, 3)

In [56]:
size(A2 .+ A3)

(2, 2, 3)

In [57]:
A2 .+ A3

2×2×3 Array{Int64, 3}:
[:, :, 1] =
 11  31
 22  42

[:, :, 2] =
 13  33
 24  44

[:, :, 3] =
 15  35
 26  46

In [58]:
(1:9) .* (1:9)'  # `(9,)` と `(1, 9)` との演算で結果は `(9, 9)` （9×9の行列）となる

9×9 Matrix{Int64}:
 1   2   3   4   5   6   7   8   9
 2   4   6   8  10  12  14  16  18
 3   6   9  12  15  18  21  24  27
 4   8  12  16  20  24  28  32  36
 5  10  15  20  25  30  35  40  45
 6  12  18  24  30  36  42  48  54
 7  14  21  28  35  42  49  56  63
 8  16  24  32  40  48  56  64  72
 9  18  27  36  45  54  63  72  81

In [59]:
binomial.(0:5, (0:5)')  # 演算子だけでなく（2引数以上の）関数でもOK

6×6 Matrix{Int64}:
 1  0   0   0  0  0
 1  1   0   0  0  0
 1  2   1   0  0  0
 1  3   3   1  0  0
 1  4   6   4  1  0
 1  5  10  10  5  1

In [60]:
zeros(2, 4) .+ zeros(4, 2)  # 一方が `1` でなければ（例え倍数でも）NG

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2 and 4

### 7-1-3. `@.` マクロ

#### コード7-10. `@.` マクロの使用例(1)

In [61]:
a = 0.0:0.5:4.0;

In [62]:
inv.(abs.(sin.(a)))

9-element Vector{Float64}:
 Inf
  2.085829642933488
  1.1883951057781212
  1.0025113042467249
  1.0997501702946164
  1.67092154555868
  7.086167395737187
  2.850763437540464
  1.3213487088109024

In [63]:
@. inv(abs(sin(a)))  # equivalent to `inv.(abs.(sin.(a)))` 

9-element Vector{Float64}:
 Inf
  2.085829642933488
  1.1883951057781212
  1.0025113042467249
  1.0997501702946164
  1.67092154555868
  7.086167395737187
  2.850763437540464
  1.3213487088109024

In [64]:
@. max(sin(a), cos(a))  # equivalent to `max.(sin.(a), cos.(a))`

9-element Vector{Float64}:
  1.0
  0.8775825618903728
  0.8414709848078965
  0.9974949866040544
  0.9092974268256817
  0.5984721441039564
  0.1411200080598672
 -0.35078322768961984
 -0.6536436208636119

In [65]:
@. sin(a * a') + 1  # equivalent to `sin.(a .* a') .+ 1`

9×9 Matrix{Float64}:
 1.0  1.0      1.0       1.0        …  1.0        1.0       1.0
 1.0  1.2474   1.47943   1.68164       1.99749    1.98399   1.9093
 1.0  1.47943  1.84147   1.99749       1.14112    0.649217  0.243198
 1.0  1.68164  1.99749   1.77807       0.0224699  0.141066  0.720585
 1.0  1.84147  1.9093    1.14112       0.720585   1.65699   1.98936
 1.0  1.94898  1.59847   0.428439   …  1.938      1.62472   0.455979
 1.0  1.99749  1.14112   0.0224699     1.41212    0.120304  0.463427
 1.0  1.98399  0.649217  0.141066      0.120304   0.688881  1.99061
 1.0  1.9093   0.243198  0.720585      0.463427   1.99061   0.712097

#### コード7-11. `@.` マクロの使用例(2)

In [66]:
A = [1 2; 3 5];
B = [2 3; 4 1];
c = [1, -1];

In [67]:
A * B .+ c

2×2 Matrix{Int64}:
 11   6
 25  13

In [68]:
@. A * B + c  # equivalent to `A .* B .+ c` NOT to `A * B .+ c`

2×2 Matrix{Int64}:
  3  7
 11  4

In [69]:
@. $(A * B) + c  # equivalent to `A * B .+ c`

2×2 Matrix{Int64}:
 11   6
 25  13

In [70]:
μ = 5.0; σ = 2.0; N = 500;

In [71]:
randn(N) .* σ .+ μ  # 実行する度に結果は変わります

500-element Vector{Float64}:
 7.794812893181122
 2.2539394225487697
 7.270426051766719
 4.9641694998807235
 5.427439965097286
 7.3901645204331246
 5.5967628964876965
 3.8471624077132636
 4.57893624667025
 6.209276914955653
 4.0043940423298094
 1.8151526778958895
 3.3842103641733217
 ⋮
 5.794005340412893
 0.6504045141573949
 5.983065791207082
 4.5405974382600744
 4.64427050016337
 0.8239353432197962
 2.266888688449491
 6.343969556457196
 5.482376813519411
 2.0969260250961717
 7.484360975500344
 5.187975018681206

In [72]:
@. randn(N) * σ + μ

LoadError: MethodError: no method matching +(::Vector{Float64}, ::Float64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:383
[0m  +([91m::LinearAlgebra.UniformScaling[39m, ::Number) at ~/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:144
[0m  ...

In [73]:
@. $randn(N) * σ + μ  # 実行する度に結果は変わります

500-element Vector{Float64}:
  4.893593155594752
  7.295216606208635
  6.377630597720385
  3.441900252679032
 10.13784226629634
  2.586488601893259
  7.599958421610714
  1.086734005542469
  8.39523063840458
  4.383344993559679
  1.311789175958383
  5.183836229167832
  6.90408727927368
  ⋮
  5.632956595530863
  4.608711444121653
  4.768199677295196
  5.353305216441186
  5.101133624560434
  1.4445722658191045
  6.062246473609266
  3.1183942486753846
  5.274791460615795
  6.429438192955287
  2.7071470475898076
  3.0334708030094713

In [74]:
@. $sum(A * B)  # same as `sum(A .* B)`

25

### 7-1-4. ブロードキャスティングの特徴

#### コード7-12. ブロードキャスティングの例(5)

In [75]:
a = [1.2, ℯ, π, √99];

In [76]:
map(x -> floor(Int, x), a)

4-element Vector{Int64}:
 1
 2
 3
 9

In [77]:
[floor(Int, x) for x in a]

4-element Vector{Int64}:
 1
 2
 3
 9

In [78]:
floor.(Int, a)

4-element Vector{Int64}:
 1
 2
 3
 9

#### コード7-13. ドット構文（ブロードキャスティング）のNG例(1)

In [79]:
a2 = [1, 2]; a3 = [3, 4, 5];

In [80]:
a2 .* a2 == map(*, a2, a2) == [1, 4]  # 要素数が同じものどうしなら結果は同じ

true

In [81]:
map(*, a2, a3)  # `map` 関数は `a2` と `a3` の要素数が異なっていても動作する

2-element Vector{Int64}:
 3
 8

In [82]:
a2 .* a3

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2 and 3

In [83]:
A = [1 2; 3 4];

In [84]:
map(*, A, a3)  # `map` 関数ならこれも動作する

3-element Vector{Int64}:
  3
 12
 10

In [85]:
A .* a3

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2 and 3

#### コード7-14. ドット構文（ブロードキャスティング）のNG例(2)

In [86]:
randnds3(N, μ, σ) = randn(N) .* σ .+ μ  # コード7-7. 参照（再掲）

randnds3 (generic function with 1 method)

In [87]:
μ = 5.0; σ = 2.0;

In [88]:
xs = randnds3(5000, μ, σ);
ys = randnds3(6000, μ, σ);

In [89]:
extrema(xs .* ys' .+ 1)  # 方法1：ブロードキャスティングを利用

(-38.82256394972482, 152.8412525705154)

In [90]:
extrema(x * y + 1 for x in xs, y in ys)  # 方法2：ジェネレータ式を利用

(-38.82256394972482, 152.8412525705154)

In [91]:
chk_extrema1(xs, ys, c) = extrema(xs .* ys' .+ c)

chk_extrema1 (generic function with 1 method)

In [92]:
chk_extrema2(xs, ys, c) = extrema(x * y + c for x in xs, y in ys)

chk_extrema2 (generic function with 1 method)

In [93]:
chk_extrema1(xs, ys, 1) == chk_extrema2(xs, ys, 1)

true

In [94]:
@time chk_extrema1(xs, ys, 1);

  0.172055 seconds (3 allocations: 228.882 MiB, 14.46% gc time)


In [95]:
@time chk_extrema2(xs, ys, 1);

  0.087048 seconds (1 allocation: 32 bytes)


In [96]:
chk_extrema1′(xs, ys, c) = @. $extrema(xs * ys' + c)

chk_extrema1′ (generic function with 1 method)

In [97]:
chk_extrema1′(xs, ys, 1) == chk_extrema1(xs, ys, 1) == chk_extrema2(xs, ys, 1)

true

In [98]:
@time chk_extrema1′(xs, ys, 1);

  0.157045 seconds (3 allocations: 228.882 MiB, 6.55% gc time)
